In [2]:
import pandas as pd # library for data analsysis
import numpy as np

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [8]:
r = requests.get('https://usc.data.socrata.com/api/geospatial/r8qd-yxsr?method=export&format=GeoJSON')
losangeles_data = r.json()

In [9]:
neighborhoods_data = losangeles_data['features']

In [12]:
neighborhoods_data[2]

{'type': 'Feature',
 'properties': {'external_i': 'agoura-hills',
  'name': 'Agoura Hills',
  'location': 'POINT(34.146736499122795 -118.75988450000015)',
  'latitude': '-118.75988450000015',
  'slug_1': None,
  'sqmi': '8.14676029818',
  'display_na': 'Agoura Hills L.A. County Neighborhood (Current)',
  'set': 'L.A. County Neighborhoods (Current)',
  'slug': 'agoura-hills',
  'longitude': '34.146736499122795',
  'name_1': None,
  'kind': 'L.A. County Neighborhood (Current)',
  'type': 'standalone-city'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-118.76192500000009, 34.16820299912263],
     [-118.72631699999998, 34.16787499912258],
     [-118.73232400000018, 34.1543019991227],
     [-118.73706000000027, 34.15435299912277],
     [-118.73702385723149, 34.14304324369543],
     [-118.73587193294368, 34.14267020794711],
     [-118.73493704578691, 34.14215345623102],
     [-118.7306383000593, 34.139239215941366],
     [-118.72918210881832, 34.13843288950915],
     [-118.727

In [13]:
neighborhoods = pd.DataFrame(columns=['Neighborhood', 'Latitude', 'Longitude'] )
neighborhoods

,Neighborhood,Latitude,Longitude


In [14]:
for data in neighborhoods_data:
    neighborhood_name = data['properties']['name']
    neighborhood_lat = data['properties']['latitude']
    neighborhood_lon = data['properties']['longitude']
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [16]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Acton,-118.16981019229348,34.497355239240846
1,Adams-Normandie,-118.30020800000011,34.031461499124156
2,Agoura Hills,-118.75988450000015,34.146736499122795
3,Agua Dulce,-118.3171036690717,34.504926999796837
4,Alhambra,-118.13651200000021,34.085538999123571


In [17]:
address = 'Los Angeles, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles, CA are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Los Angeles, CA are 34.0536909, -118.2427666.


## The values in the JSON file are all around 34, -118. However, latitude and longitude are reversed, so we will change the column names to reflect this.

In [31]:
neighborhoods.rename(columns = {'Latitude':'Longitude', 'Longitude':'Latitude'}, inplace=True)
neighborhoods.head()

,Neighborhood,Longitude,Latitude
0,Acton,-118.16981019229348,34.497355239240846
1,Adams-Normandie,-118.30020800000011,34.031461499124156
2,Agoura Hills,-118.75988450000015,34.146736499122795
3,Agua Dulce,-118.3171036690717,34.504926999796837
4,Alhambra,-118.13651200000021,34.085538999123571


In [32]:
neighborhoods.shape

(272, 3)

In [33]:
neighborhoods.dropna(subset=['Latitude'], inplace=True)
neighborhoods.dropna(subset=['Longitude'], inplace=True)
neighborhoods.shape

(272, 3)

In [24]:
map_los_angeles = folium.Map(location=[latitude, longitude], zoom_start=10)
map_los_angeles

In [34]:
for lat, lng, in zip(neighborhoods['Latitude'], neighborhoods['Longitude']):
    map_los_angeles.add_child(
        folium.features.CircleMarker(
            [float(lat), float(lng)],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )
    
map_los_angeles

# Foursquare API

In [35]:
CLIENT_ID = 'ERFCS0HI2ODMI5PVVGK2FXA0BBDO5245K3YHLDTXF4UBWPBG' # your Foursquare ID
CLIENT_SECRET = 'RP2Z2TV0DWKA10HHRQI5YYI5LCVBYTWTQHNMQ41PXAN2BRM0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ERFCS0HI2ODMI5PVVGK2FXA0BBDO5245K3YHLDTXF4UBWPBG
CLIENT_SECRET:RP2Z2TV0DWKA10HHRQI5YYI5LCVBYTWTQHNMQ41PXAN2BRM0


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
LIMIT=100

los_angeles_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Alondra Park
Artesia
Altadena
Angeles Crest
Arcadia
Arleta
Arlington Heights
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Vermont-Slauson
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Green Valley
Bell
Beverly Crest
Beverly Grove
Burbank
Koreatown
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brentwood
Broadway-Manchester
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Chatsworth
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Northridge
Commerce
Compton
Cypress Park
La Mirada
Covina
Cudahy
Culver City
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East La Mirada
Elizabeth Lake
East Los Angeles
East Pasadena
East San Gabriel
Echo Park
El Monte
El Segundo
El Sereno
Elysian Park
Elysian Valley
Vermont Square
Encino
Exposition Park
Fairfax
Flo

In [39]:
print(los_angeles_venues.shape)
los_angeles_venues.head()

(2989, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355239240846,-118.16981019229348,Epik Engineering,34.498718,-118.168046,Construction & Landscaping
1,Acton,34.497355239240846,-118.16981019229348,Alma Gardening Co.,34.494762,-118.172550,Construction & Landscaping
2,Adams-Normandie,34.031461499124156,-118.30020800000011,Orange Door Sushi,34.032485,-118.299368,Sushi Restaurant
3,Adams-Normandie,34.031461499124156,-118.30020800000011,Shell,34.033095,-118.300025,Gas Station
4,Adams-Normandie,34.031461499124156,-118.30020800000011,Sushi Delight,34.032445,-118.299525,Sushi Restaurant


In [40]:
los_angeles_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton,2,2,2,2,2,2
Adams-Normandie,9,9,9,9,9,9
Agoura Hills,30,30,30,30,30,30
Agua Dulce,1,1,1,1,1,1
Alhambra,13,13,13,13,13,13
Alondra Park,4,4,4,4,4,4
Altadena,5,5,5,5,5,5
Arcadia,21,21,21,21,21,21
Arleta,4,4,4,4,4,4


In [41]:
print('There are {} uniques categories.'.format(len(los_angeles_venues['Venue Category'].unique())))

There are 322 uniques categories.


# Analyzing the Data

#### Frequency of each venue category.
We use one-hot encoding for this.

In [42]:
los_angeles_onehot = pd.get_dummies(los_angeles_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
los_angeles_onehot['Neighborhood'] = los_angeles_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [los_angeles_onehot.columns[-1]] + list(los_angeles_onehot.columns[:-1])
los_angeles_onehot = los_angeles_onehot[fixed_columns]

los_angeles_onehot.head()

,Yoga Studio,ATM,Accessories Store,Airport,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
los_angeles_onehot.shape

(2989, 322)

#### Grouping by neighborhood.
We calculate the mean of each frequency.

In [45]:
los_angeles_grouped = los_angeles_onehot.groupby('Neighborhood').mean().reset_index()
los_angeles_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Airport,Alternative Healer,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Warehouse Store,Watch Shop,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Acton,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adams-Normandie,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Agoura Hills,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Agua Dulce,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alhambra,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.076923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
los_angeles_grouped.shape

(241, 322)

In [47]:
num_top_venues = 5
for hood in los_angeles_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = los_angeles_grouped[los_angeles_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acton----
                           venue  freq
0     Construction & Landscaping   1.0
1          Performing Arts Venue   0.0
2                           Park   0.0
3  Paper / Office Supplies Store   0.0
4              Other Repair Shop   0.0


----Adams-Normandie----
              venue  freq
0  Sushi Restaurant  0.33
1              Park  0.11
2        Playground  0.11
3     Grocery Store  0.11
4       Gas Station  0.11


----Agoura Hills----
                  venue  freq
0  Fast Food Restaurant  0.13
1                Bakery  0.07
2    Chinese Restaurant  0.07
3     Indian Restaurant  0.07
4        Breakfast Spot  0.07


----Agua Dulce----
                           venue  freq
0                        Airport   1.0
1                    Yoga Studio   0.0
2                         Notary   0.0
3                           Park   0.0
4  Paper / Office Supplies Store   0.0


----Alhambra----
                 venue  freq
0    Convenience Store  0.31
1  Sporting Goods Shop  0.08
2     

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
la_neighborhoods_sorted = pd.DataFrame(columns=columns)
la_neighborhoods_sorted['Neighborhood'] = los_angeles_grouped['Neighborhood']

for ind in np.arange(los_angeles_grouped.shape[0]):
    la_neighborhoods_sorted.iloc[ind, 1:] = return_most_common_venues(los_angeles_grouped.iloc[ind, :], num_top_venues)

la_neighborhoods_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Construction & Landscaping,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
1,Adams-Normandie,Sushi Restaurant,Gas Station,Park,Playground,Locksmith,Grocery Store,Taco Place,Women's Store,Event Service,English Restaurant
2,Agoura Hills,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Indian Restaurant,Bakery,Café,Gym,Sushi Restaurant,Gas Station,Lounge
3,Agua Dulce,Airport,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
4,Alhambra,Convenience Store,Sporting Goods Shop,Video Store,Mexican Restaurant,Breakfast Spot,Fast Food Restaurant,Pet Store,Hardware Store,Bagel Shop,Pizza Place


In [72]:
kclusters = 15
laclustering = los_angeles_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(laclustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([ 7,  1,  1, 14,  0,  9,  3,  1,  1,  1], dtype=int32)

In [78]:
# add clustering labels
la_neighborhoods_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
la_merged = neighborhoods
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
la_merged = la_merged.join(la_neighborhoods_sorted.set_index('Neighborhood'), on='Neighborhood')
la_merged.head() # check the last columns!

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,-118.16981019229348,34.497355239240846,7.0,Construction & Landscaping,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
1,Adams-Normandie,-118.30020800000011,34.031461499124156,1.0,Sushi Restaurant,Gas Station,Park,Playground,Locksmith,Grocery Store,Taco Place,Women's Store,Event Service,English Restaurant
2,Agoura Hills,-118.75988450000015,34.146736499122795,1.0,Fast Food Restaurant,Chinese Restaurant,Breakfast Spot,Indian Restaurant,Bakery,Café,Gym,Sushi Restaurant,Gas Station,Lounge
3,Agua Dulce,-118.3171036690717,34.504926999796837,14.0,Airport,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
4,Alhambra,-118.13651200000021,34.085538999123571,0.0,Convenience Store,Sporting Goods Shop,Video Store,Mexican Restaurant,Breakfast Spot,Fast Food Restaurant,Pet Store,Hardware Store,Bagel Shop,Pizza Place


In [79]:
la_merged = la_merged.fillna(0)
la_merged['Cluster Labels'] = la_merged['Cluster Labels'].astype(np.int64)
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged['Latitude'], la_merged['Longitude'], la_merged['Neighborhood'], la_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [80]:
la_merged.loc[la_merged['Cluster Labels'] == 0, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,-118.13651200000021,Sporting Goods Shop,Video Store,Mexican Restaurant,Breakfast Spot,Fast Food Restaurant,Pet Store,Hardware Store,Bagel Shop,Pizza Place
8,-117.9223952817848,0,0,0,0,0,0,0,0,0
12,-118.30465647554277,0,0,0,0,0,0,0,0,0
14,-118.32733223477572,American Restaurant,Mexican Restaurant,Playground,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
15,-118.0012614768012,Fast Food Restaurant,Grocery Store,Bakery,Liquor Store,Diner,Pizza Place,Discount Store,Taco Place,Women's Store
19,-117.9751905000002,Italian Restaurant,Mexican Restaurant,Liquor Store,Thrift / Vintage Store,Grocery Store,Latin American Restaurant,Disc Golf,English Restaurant,Design Studio
21,-118.12903150000017,Sandwich Place,Pharmacy,Clothing Store,Supermarket,Chinese Restaurant,Mobile Phone Shop,Fast Food Restaurant,Mexican Restaurant,Grocery Store
22,-118.1499360000002,Sporting Goods Shop,Seafood Restaurant,Fast Food Restaurant,Park,Rental Service,Mexican Restaurant,Convenience Store,New American Restaurant,Fried Chicken Joint
31,-118.21078750000007,Bakery,Pool,Donut Shop,Fast Food Restaurant,Japanese Restaurant,Café,Fabric Shop,English Restaurant,Entertainment Service
36,-118.60151949999999,Sports Bar,Liquor Store,Garden Center,Sushi Restaurant,Restaurant,Food,Ice Cream Shop,Fast Food Restaurant,Filipino Restaurant


In [81]:
la_merged.loc[la_merged['Cluster Labels'] == 1, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,-118.30020800000011,Gas Station,Park,Playground,Locksmith,Grocery Store,Taco Place,Women's Store,Event Service,English Restaurant
2,-118.75988450000015,Chinese Restaurant,Breakfast Spot,Indian Restaurant,Bakery,Café,Gym,Sushi Restaurant,Gas Station,Lounge
6,-118.08010100000017,Chinese Restaurant,Bubble Tea Shop,Vietnamese Restaurant,Thai Restaurant,BBQ Joint,Café,Pizza Place,Shopping Mall,Boutique
9,-118.03041899311202,Gym,Football Stadium,Gymnastics Gym,Food & Drink Shop,Fast Food Restaurant,Mexican Restaurant,Shabu-Shabu Restaurant,Chinese Restaurant,Baseball Field
10,-118.4307575,Video Store,Convenience Store,Bakery,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant
11,-118.3234085,Donut Shop,Latin American Restaurant,Karaoke Bar,Vegetarian / Vegan Restaurant,Bus Station,Intersection,Liquor Store,Bakery,Fried Chicken Joint
13,-118.26237347966236,Bar,Bakery,Bank,Fried Chicken Joint,Coffee Shop,Juice Bar,Grocery Store,Mediterranean Restaurant,Fast Food Restaurant
16,-117.91246849999999,Liquor Store,Big Box Store,Pharmacy,Pizza Place,Coffee Shop,Fabric Shop,English Restaurant,Entertainment Service,Ethiopian Restaurant
17,-118.29035750000011,Clothing Store,Check Cashing Service,Mobile Phone Shop,Sandwich Place,Grocery Store,Organic Grocery,Chinese Restaurant,Women's Store,Event Service
20,-118.45841550000007,Football Stadium,Bakery,Food,Flower Shop,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor


In [82]:
la_merged.loc[la_merged['Cluster Labels'] == 2, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,-118.2449270000002,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
161,-118.21900650000001,Garden,Park,Women's Store,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
191,-118.35980750108605,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
211,-118.04883650000005,Gourmet Shop,Other Repair Shop,Coffee Shop,Eye Doctor,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
235,-118.2818,Trail,Women's Store,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor
241,-118.77026791367985,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop


In [83]:
la_merged.loc[la_merged['Cluster Labels'] == 3, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,-118.13623898201556,Food,Home Service,Pharmacy,Campground,Women's Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
23,-118.41421658346756,Home Service,Playground,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor
24,-118.17916600000018,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
85,-118.24192048907494,Clothing Store,Grocery Store,Alternative Healer,Music Venue,Falafel Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
171,-118.41890850000007,Breakfast Spot,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
247,-118.20343200000011,Furniture / Home Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
255,-118.63539850000006,Italian Restaurant,Theater,Grocery Store,Park,Martial Arts Dojo,Event Service,Flea Market,Empanada Restaurant,English Restaurant


In [84]:
la_merged.loc[la_merged['Cluster Labels'] == 4, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,-117.84178450000023,Mountain,Department Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
135,-117.7745250000001,Music Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
155,-118.45716250000007,Church,Women's Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
160,-118.19427049999999,Lake,Women's Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
226,-118.31202250000007,Park,Department Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
248,-118.45725850000031,Food Truck,Garden Center,Falafel Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
262,-117.83509899236435,Other Repair Shop,Deli / Bodega,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
266,-118.06118827749373,Lake,Women's Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop


In [85]:
la_merged.loc[la_merged['Cluster Labels'] == 5, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
108,-118.65705600000015,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
125,-117.95562783562224,Women's Store,Falafel Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop
156,-118.13161942411986,Business Service,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop


In [86]:
la_merged.loc[la_merged['Cluster Labels'] == 6, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
168,-118.02918914224676,Women's Store,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant


In [87]:
la_merged.loc[la_merged['Cluster Labels'] == 7, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,-118.16981019229348,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
94,-118.50767050010745,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
104,-118.66021608592953,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
128,-117.83603921804665,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant


In [88]:
la_merged.loc[la_merged['Cluster Labels'] == 8, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
123,-118.37410748609757,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
174,-118.10809351531806,Food Court,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant
198,-117.81326850000013,Lawyer,Women's Store,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop


In [89]:
la_merged.loc[la_merged['Cluster Labels'] == 9, la_merged.columns[[1] + list(range(5, la_merged.shape[1]))]]

,Longitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,-118.33515598608159,Home Service,Gym / Fitness Center,Park,Women's Store,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant
18,-118.35774600000005,Home Service,Park,Women's Store,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
30,-118.39398199999999,Park,Food Truck,Business Service,Boutique,Women's Store,Farm,Entertainment Service,Ethiopian Restaurant,Event Service
42,-118.24721250000012,Donut Shop,Mexican Restaurant,Kitchen Supply Store,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
45,-117.85751998453503,Video Store,Business Service,Moving Target,Dessert Shop,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Eye Doctor
55,-118.22415649999999,Mexican Restaurant,Discount Store,Bakery,Fish & Chips Shop,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant
72,-118.1596383526979,Park,Latin American Restaurant,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Light Rail Station,Southern / Soul Food Restaurant,Filipino Restaurant,Farmers Market
81,-118.29880750000001,Moving Target,Shop & Service,Stationery Store,Park,Liquor Store,Fabric Shop,Entertainment Service,Ethiopian Restaurant,Event Service
96,-117.97508050764819,Scenic Lookout,Weight Loss Center,Bus Station,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service
99,-118.29516400000006,Spanish Restaurant,Home Service,Pedestrian Plaza,Mexican Restaurant,Fabric Shop,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant
